# Recreation de la question 3 du devoir 1

## IMPORTS

In [20]:
from pickletools import optimize
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import os
from torchvision import datasets, transforms
import torchvision
import torch.utils.data
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
torch.manual_seed(6307)

from inspect import _void
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import random as rand
import math
import time
from tqdm import tqdm

## Donnees de test

In [21]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=28,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=28,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
def stochastic_rounding_t(x): 
    r = torch.rand(x.shape)
    r = r.to(device)
    d = 1-(x % 1)
    k1 = r > d
    k2 = k1 == False
    upper = torch.ceil(x*k1)
    lower = torch.floor(x*k2)
    x_int = upper+lower
    return x_int

In [23]:
def quntize_block(x, s, z, b, stochastic=False):
    c = (2**b)-1
    xc = torch.zeros(x.shape)
    if(stochastic):
        x_int = stochastic_rounding_t((x/s)+z)
        x_int[x_int<0]=0
        x_int[x_int>c]=c
        x_hat = s*(x_int-z)
    else:  
        x_int = (x/s)
        x_int = torch.round(x_int)+z
        x_int[x_int<0]=0
        x_int[x_int>c]=c
        x_hat = s*(x_int-z)
    return x_hat

def ACIQ(param, bit):
    ex = torch.mean(param)
    b = param - ex
    b = torch.abs(b)
    b = torch.mean(b)
    if(bit == 2):
        a = 2.83*b
    elif(bit == 3):
        a = 3.86*b
    elif(bit == 4):
        a = 5.03*b
    else:
        raise Exception("Seul les encodages à 2,3 ou 4 bits sont supportés")
    return a.item()

def MSE(x,x_hat):
    soustraction = x-x_hat
    carre = torch.square(soustraction)
    mse = torch.mean(carre)
    return mse.item()

def szfinder(q_min, q_max, b):
    s = (q_max-q_min)/((2**b)-1)
    #if(s==0):
        #s = 0.001
    z = (-q_min)/s
    return s,z

def bruteForceOptim(x, b, step):
    q_min = torch.min(x)
    q_max = torch.max(x)
    q_min = q_min.item()
    q_max = q_max.item()
    mse_min = np.array([np.inf, q_min])
    mse_max =np.array([np.inf, q_max])
    stop = torch.mean(x)
    stop = stop.item()
    while(q_max >= stop):
        s, z = szfinder(q_min, q_max, b)
        x_hat = quntize_block(x, s, z, b, stochastic=False)
        mse_t = MSE(x,x_hat)
        if(mse_t < mse_max[0]):
            mse_max[0] = mse_t
            mse_max[1] = q_max
        q_max -= step
    while(q_min <= x.mean()):
        s, z = szfinder(q_min, q_max, b)
        x_hat = quntize_block(x, s, z, b, stochastic=False)
        mse_t = MSE(x,x_hat)
        if(mse_t < mse_max[0]):
            mse_min[0] = mse_t
            mse_min[1] = q_max
        q_min += step
    return mse_min[1], mse_max[1]

def clipping_range(x,bit, step, method="min_max"):
    if(method=="ACIQ"):
        a = ACIQ(x, bit)
        q_min = -a
        q_max = a
    
    elif(method == "heuristic"):
        q_min, q_max = bruteForceOptim(x, bit, step)
    else:
        q_min = torch.min(x)
        q_max = torch.max(x)
        q_min = q_min.item()
        q_max = q_max.item()
    return q_min,q_max

In [24]:
def calibration(model,device, calibration_loader=torch.utils.data.dataloader.DataLoader):
    model.eval()
    with torch.no_grad():
      bit = 3
      q = np.array([[0,0],[0,0],[0,0]])
      quantization = [False, False]
      methode = "min_max"
      for batch_idx, (x, y) in enumerate(calibration_loader):
        x = x.to(device=device)
        y = y.to(device=device)
        scores, a1, a2, a3 = model(x, bit, quantization, methode, q )
      return a1, a2, a3

def weight_quantization(param, bit, clipping_method, filter_wise=False):
    param_hat = torch.zeros_like(param)
    if(filter_wise):
        param_hat = torch.ones(param.shape)
        for i in range(param.shape[0]):
            x = param[i]
            q_min, q_max = clipping_range(x,bit,0.1, clipping_method)
            s,z = szfinder(q_min,q_max,bit)
            if(s == 0):
                s = 0.01
            param_hat[i] = quntize_block(x, s, z, bit, stochastic=False)
            
    else:
        q_min, q_max = clipping_range(param,bit,0.1, clipping_method)
        s,z = szfinder(q_min,q_max,bit)
        param_hat = quntize_block(param, s, z, bit, stochastic=False)
    return param_hat

def npoids(x, bit,methode="min_max", filter_wise = False):
    with torch.no_grad():
        k1 = weight_quantization(x, bit, methode, filter_wise)
        k1 = k1.to(device)
        x1 = nn.Parameter(k1)
        x = x1
        return x


def activation_quantization(activation, bit, q_min, q_max, filter_wise=False):
    activation_hat = torch.zeros_like(activation)
    if(filter_wise):
        activation_hat = torch.ones(activation.shape)
        for i in range(activation.shape[0]):
            s,z = szfinder(q_min,q_max,bit)
            activation_hat = quntize_block(activation, s, z, bit, stochastic=False)
    else:
        s,z = szfinder(q_min,q_max,bit)

        activation_hat = quntize_block(activation, s, z, bit, stochastic=False)
    return activation_hat

In [25]:
class QLinear(nn.Linear):
    #def __init__(self, in_features, out_features,q_min, q_max, bit, methode="min_max"  ,quantization = [False, False], bias=True,
                 #w_qmodule=nn.Identity(), act_qmodule=nn.Identity()
                 #):
    def __init__(self, in_features, out_features,q_min, q_max, bias=True,
                 w_qmodule=nn.Identity(), act_qmodule=nn.Identity()
                 ):
        super(QLinear, self).__init__(in_features, out_features, bias)
        self.w_quant = w_qmodule
        self.act_quant = act_qmodule
        self.q_min = q_min
        self.q_max = q_max

    def forward(self, input, bit, quantization, methode, q_min, q_max, filter):
      #TODO
      self.quantization = quantization
      self.q_min = q_min
      self.q_max = q_max
      # Quantize the weight
      weight_q = self.weight
      input_q = input
      if(not self.training):
        if(self.quantization[0]):
          weight_q = npoids(weight_q, bit,methode,True)
          self.weight = weight_q
        if(self.quantization[1]):
          input_q = activation_quantization(input, bit, self.q_min, self.q_max, filter_wise=filter)

        # Quantize the Activation
      output = F.linear(input_q, weight_q, self.bias)

      return output
    
class QConv2d(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, q_min, q_max, 
                 stride=1, padding=0, dilation=1, groups=1, bias=True,
                 w_qmodule=nn.Identity(), act_qmodule=nn.Identity()
                 ):
        super(QConv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            groups, bias)

        self.w_quant = w_qmodule
        self.act_quant = act_qmodule
        self.q_min = q_min
        self.q_max = q_max

    def forward(self, input,bit, quantization, methode, q_min, q_max, filter):
        self.quantization = quantization
        self.q_min = q_min
        self.q_max = q_max


        #TODO
        # Quantize the weight
        weight_q = self.weight
        input_q = input
        if(not self.training):
          if(self.quantization[0]):
            weight_q = npoids(weight_q, bit,methode,True)
            self.weight = weight_q
          if(self.quantization[1]):
            input_q = activation_quantization(input, bit, self.q_min, self.q_max, filter_wise=filter).to(device)
            #print(input_q)
        # Quantize the Activation
        

        output = F.conv2d(input_q, weight_q, self.bias, self.stride, self.padding, self.dilation, self.groups)

        return output

In [26]:
class Quantized_VGG(nn.Module):
    def __init__(self, w_bit, clipping_method, filter_wise):
        super(Quantized_VGG, self).__init__()
        
        self.w_bit = w_bit
        self.clipping_method = clipping_method
        self.filter_wise = filter_wise
        self.stochastic = False
        
        self.conv1_1 = QConv2d(in_channels=3, out_channels=64, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv1_2 = QConv2d(in_channels=64, out_channels=64, kernel_size=3,q_min = -1, q_max = 1, padding=1)

        self.conv2_1 = QConv2d(in_channels=64, out_channels=128, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv2_2 = QConv2d(in_channels=128, out_channels=128, kernel_size=3,q_min = -1, q_max = 1, padding=1)

        self.conv3_1 = QConv2d(in_channels=128, out_channels=256, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv3_2 = QConv2d(in_channels=256, out_channels=256, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv3_3 = QConv2d(in_channels=256, out_channels=256, kernel_size=3,q_min = -1, q_max = 1, padding=1)

        self.conv4_1 = QConv2d(in_channels=256, out_channels=512, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv4_2 = QConv2d(in_channels=512, out_channels=512, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv4_3 = QConv2d(in_channels=512, out_channels=512, kernel_size=3,q_min = -1, q_max = 1, padding=1)

        self.conv5_1 = QConv2d(in_channels=512, out_channels=512, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv5_2 = QConv2d(in_channels=512, out_channels=512, kernel_size=3,q_min = -1, q_max = 1, padding=1)
        self.conv5_3 = QConv2d(in_channels=512, out_channels=512, kernel_size=3,q_min = -1, q_max = 1, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = QLinear(25088, 4096,q_min = -1, q_max = 1)
        self.fc2 = QLinear(4096, 4096, q_min = -1, q_max = 1)
        self.fc3 = QLinear(4096, 10,q_min = -1, q_max = 1)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4, weight_decay=5e-7)

    
    def forward(self, x, bit, quantization, methode, filter = False):
        
        x = F.relu(self.conv1_1(x, bit[0], quantization = [quantization[0], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv1_2(x, bit[1], quantization = [quantization[1], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x, bit[2], quantization = [quantization[2], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv2_2(x, bit[3], quantization = [quantization[3], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x, bit[4], quantization = [quantization[4], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv3_2(x, bit[5], quantization = [quantization[5], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv3_3(x, bit[6], quantization = [quantization[6], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x, bit[7], quantization = [quantization[7], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv4_2(x, bit[8], quantization = [quantization[8], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv4_3(x, bit[9], quantization = [quantization[9], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x, bit[10], quantization = [quantization[10], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv5_2(x, bit[11], quantization = [quantization[11], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = F.relu(self.conv5_3(x, bit[12], quantization = [quantization[12], False], methode=methode,  q_min = -1,q_max =  1,filter = filter))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x, bit[13], quantization = [quantization[13], False], methode=methode,  q_min = -1,q_max = 1,filter = filter))
        x = F.relu(self.fc2(x, bit[14], quantization = [quantization[14], False], methode=methode,  q_min = -1,q_max = 1,filter = filter))
        out = self.fc3(x, bit[15], quantization = [quantization[15], False], methode=methode,  q_min = -1,q_max = 1,filter = filter)
        
        return out

    
    def fit(self, nb_epochs=5, trainloader=torch.utils.data.dataloader.DataLoader):
        err_train = np.zeros(nb_epochs) 
        k = [32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32]
        w = [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
        criterion = nn.CrossEntropyLoss()
        learning_rate = 0.001
        optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        for i in range(nb_epochs):
            for batch_idx, (data, targets) in enumerate(trainloader):
                data = data.to(device=device)
                targets = targets.to(device=device)
                scores = self(data, k, w, "min_max", filter = False)
                loss = criterion(scores, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                err_train[i] = loss.data
            print('Epoch:', i, 'Train loss = ',  err_train[i])

    def test_acc(self,test_size, bit, quantization, methode, valloader=torch.utils.data.dataloader.DataLoader, filter = False):
        self.filter = filter
        num_correct = 0
        num_samples = 0
        self.quantization = quantization
        self.bit = bit
        self.methode = methode
        self.eval()
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(valloader):
                if(batch_idx % 10 == 0):
                    a = batch_idx/len(valloader)
                x = x.to(device=device)
                y = y.to(device=device)
                scores = self(x, self.bit, self.quantization, self.methode, self.filter)
                _, predictions = scores.max(1)
                num_correct += (predictions == y).sum()
                num_samples += predictions.size(0)
                if(batch_idx == test_size):
                    break
            self.train()
            test = num_correct/num_samples*100
        return test

    def model_size(self, bit, quantization_map):
        size = 0
        i = 0
        for idx, (name, param) in enumerate(self.named_parameters()):
            if(idx % 2 == 0):
                a = torch.prod(torch.tensor(param.shape))
                if(quantization_map[i]):
                    sizet = bit[i]*a
                else:
                    sizet = 32*a
                i+=1 
                size += sizet
                
        return size/(2**20) 

    def plot_weight_hist(self,quantization=False):
        sns.set()
        plt.figure(figsize=(6,4))
        for i, p in  enumerate(self.named_parameters()):
            name, param = p
            w = param.cpu().detach().numpy().reshape(-1)
            plt.title("Histogram of Weights")
            plt.hist(w, density=True, alpha=0.4)
        plt.legend()
        plt.show()

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = Quantized_VGG(w_bit = 8, clipping_method="min_max", filter_wise=False)
network.to(device)
network.load_state_dict(torch.load("VGG16_CIFAR10", map_location=torch.device(device)))

<All keys matched successfully>

Les fonctions qui suivent permettent de donner un score pondere aux results

In [28]:
def evaluation(mass_result, mass_weight, result, weight):
    return mass_result*result+mass_weight*((1/weight)*1000)

def random_bitmap(size):
    bit = np.random.randint(2, size=(1, size))+3
    quantization_map = np.random.randint(2, size=(1, size))
    return bit[0], quantization_map[0]

Ici, YAML_generator modifie le fichier d'architecture de Eyeriss pour mettre la valeur desiree. energy_compute_read permet de lire le fichier de output de mapper pour lire l'energie optimale. 

In [32]:
def YAML_generator(bit_width = 16, path="eyeriss_base.yaml"):
    width = 4*bit_width
    with open(path,'r') as file:
        txt = file.read()
        txt_new = txt.replace('$bit_width$',str(bit_width))
        txt_new = txt_new.replace('$m_width$',str(width))
        txt_new = txt_new.replace('$m_width2$',str(bit_width))
        with open('arch/eyeriss_like.yaml','w') as output:
            output.write(txt_new)

def energy_compute_read(path = "timeloop-mapper.stats.txt"):
    with open(path) as f:
        lines = f.readlines()
    f.close
    i = 0
    for word in lines[len(lines)-2]:
        i += 1
        if word == "=":
            break
    s = float(lines[len(lines)-2][i:len(lines[len(lines)-2])-1])
    return(s)

get_score_energy permet de sortir la somme des energies de toutes les couches

In [33]:
def get_score_energy(array = np.ones((1,16)), bit = np.ones((1,16))):
    energy = 0
    inarray = (array == 0)*16+(bit*array)
    for v in range(1,14):
        YAML_generator(inarray[v], path="eyeriss_base.yaml")
        mapper_command = f"timeloop-mapper prob/VGG02_layer{v}.yaml \
                arch/components/*.yaml \
                arch/eyeriss_like.yaml \
                constraints/*.yaml     \
                mapper/mapper.yaml" + ">/dev/null 2>&1"
        os.system(mapper_command)
        x = energy_compute_read()
        energy += x 
    return energy

## Simulation

Ici, une ponderation de 3 pour 1 en faveur de la consommation d'energie a ete implementee

In [35]:
bit, quantization_map = random_bitmap(16)
test_size = 30
loop_size = 40
mass_result = 1
mass_weight = 3
resultats = np.array([-np.inf, bit, quantization_map, np.inf, np.inf, np.inf])
for i in tqdm(range(loop_size)):
    network.load_state_dict(torch.load("VGG16_CIFAR10", map_location=torch.device(device)))
    result1 = network.test_acc(test_size, bit = bit, quantization = quantization_map, methode = "min_max", valloader = testloader) #Score acc
    if result1 < 85:
        weight = np.inf
        print(f"Pass: {result1}")
    else:
        weight = get_score_energy(quantization_map, bit)
    score = evaluation(mass_result, mass_weight, result1, weight)
    if(result1 > 85 and score > resultats[0]):
        resultats[0] = score
        resultats[1] = bit
        resultats[2] = quantization_map
        resultats[3] = weight
        resultats[4] = result1
        print(result1)
        print(resultats[0])
        print(resultats[1])
        print(resultats[2])
        print(resultats[3])
        print(resultats[4])
    bit, quantization_map = random_bitmap(16)

/tmp/ipykernel_219954/3836842984.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  resultats = np.array([-np.inf, bit, quantization_map, np.inf, np.inf, np.inf])
  2%|█                                           | 1/40 [01:14<48:31, 74.66s/it]

Pass: 83.7557601928711


  5%|██▏                                         | 2/40 [02:27<46:47, 73.87s/it]

Pass: 72.58065032958984


  8%|███▎                                        | 3/40 [03:37<44:24, 72.01s/it]

...

Pass: 82.83409881591797


 98%|█████████████████████████████████████████▉ | 39/40 [53:05<01:20, 80.62s/it]

Pass: 72.69585418701172


100%|███████████████████████████████████████████| 40/40 [54:06<00:00, 81.16s/it]

Pass: 84.9078369140625


In [36]:
inarray = (resultats[2] == 0)*16+(resultats[1]*resultats[2])
print(f"La meilleure combinaison trouvee est: {inarray}")
print(f"Cela donne une precision de {resultats[4]} %")
print(f"La somme des energies par cycle de toutes les couches est: {resultats[3]} pJ/Comput")

La meilleure combinaison trouvee est: [16  4  4  4  3  4 16  4 16  3  3 16 16  4 16  4]
Cela donne une precision de 85.59907531738281 %
La somme des energies par cycle de toutes les couches est: 49.22 pJ/Comput


In [37]:
def get_score_energy(array = np.ones((1,16)), bit = np.ones((1,16))):
    energy = 0
    list_energy = []
    inarray = (array == 0)*16+(bit*array)
    for v in range(1,14):
        YAML_generator(inarray[v], path="eyeriss_base.yaml")
        mapper_command = f"timeloop-mapper prob/VGG02_layer{v}.yaml \
                arch/components/*.yaml \
                arch/eyeriss_like.yaml \
                constraints/*.yaml     \
                mapper/mapper.yaml" + ">/dev/null 2>&1"
        os.system(mapper_command)
        x = energy_compute_read()
        list_energy.append(x)   
        print(v)
        energy += x 
    return energy, list_energy

In [42]:
b = inarray
q = resultats[2]

In [43]:
get_score_energy(q, b)

1
2
3
4
5
6
7
8
9
10
11
12
13


(49.22,
 [3.63, 2.51, 2.37, 2.16, 2.33, 5.76, 3.25, 5.97, 2.17, 2.55, 6.71, 6.71, 3.1])